1. Name - Humera Bano
2. Github Username - humera811
3. USC ID - 7255691039

In [432]:
# organizing all imports together

import pandas as pd
import os
import glob
import numpy as np
import re
import math
import warnings
import csv
from os.path import join, getsize
from pathlib import Path



## __Time Series Classification__
Part 1: Feature Creation/Extraction
An interesting task in machine learning is classification of time series. In this problem,
we will classify the activities of humans based on time series obtained by a Wireless
Sensor Network.

#### __(a) Download the AReM data from: https://archive.ics.uci.edu/ml/datasets/ Activity+Recognition+system+based+on+Multisensor+data+fusion+\%28AReM\ %29 . The dataset contains 7 folders that represent seven types of activities. In each folder, there are multiple files each of which represents an instant of a human performing an activity.1 Each file containis 6 time series collected from activities of the same person, which are called avg rss12, var rss12, avg rss13, var rss13, vg rss23, and ar rss23. There are 88 instances in the dataset, each of which contains 6 time series and each time series has 480 consecutive values.__

In [433]:
# removing extra comma in cycling dataset9.csv and cycling dataset14.csv manually as suggested on piazza
# bending2 dataset4.csv space separated (delimitter/sep_)
# https://www.geeksforgeeks.org/how-to-use-glob-function-to-find-files-recursively-in-python/
csv_files = glob.glob('../../data/AReM/*/*.csv', recursive=True)

# print(csv_files)
path = "../../data/AReM/"
actv_dir = ['bending1', 'bending2']
actv_dir2 = ['bending1','bending2','cycling', 'lying', 'sitting', 'standing', 'walking']
columns = ['# Columns: time', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']


#### __(b) Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1, 2, and 3 in other folders as test data and other datasets as train data.__

In [482]:
# https://stackoverflow.com/questions/22809061/read-space-separated-data-with-pandas
# https://www.summet.com/dmsi/html/CSVdataFiles.html
# https://pandas.pydata.org/pandas-docs/version/0.25.0/reference/api/pandas.DataFrame.T.html
# https://docs.python.org/3/library/os.html
# https://www.tutorialspoint.com/python/os_walk.htm
# https://www.geeksforgeeks.org/python-os-path-splitext-method/

csv_files = glob.glob('../../data/AReM/*/*.csv')
# print(csv_files)
# !/usr/bin/python
def train_or_test(root):
    trainset = []
    testset = []
    # scanning bottom-to-up
    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            # Split the path in root and ext pair
            root_ext = os.path.splitext(name)
            if (root_ext[1].endswith('.csv')):
                # https://www.geeksforgeeks.org/python-regex-re-search-vs-re-findall/
                file_path = os.path.join(root, name)
                # print(file_path)
                my_path = os.path.split(root)
                feature_act = my_path[-1]
                # my_path = Path(path)
                # activity = my_path.stem
                regex = '\d+'
                match = re.findall(regex, name)
                # for bending 1 and 2
                match_activity = int(match[0])
                if (match_activity <= 2 or (feature_act not in ["bending1", "bending2"] and match_activity == 3)):
                    testset.append(file_path)
                else:
                    trainset.append(file_path)
    return trainset, testset

### __(c) Feature Extraction Classification of time series usually needs extracting features from them.In this problem, we focus on time-domain features__


#### __i. Research what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc)__

Time-domain features are usually used in time series classification are:
1. Means
2. Standard Deviations
3. Skewness, Kurtosis
4. Maximum and minimum values

#### __ii. Extract the time-domain features minimum, maximum, mean, median, standard deviation, first quartile, and third quartile for all of the 6 time series in each instance.You are free to normalize/standardize features or use them directly__

In [487]:
def load_data(files, feature, columns, segs=1, standard=False):
    import csv
    sniffer = csv.Sniffer()
    
    # generate columns of describes and label with indecies
    describe_order = ['mean', 'std', 'min', '_1stquart_', 'median', '_3rdquart_','max']
    desc_order_idx = [desc + str(i) for i in range(1, 7) for desc in describe_order]
    
    instances  = [] # train and test instances
    labels = []
    for csv in files:
        # get the label of the instance read from current csv
        parent_folder = os.path.dirname(csv)
        activity = os.path.split(parent_folder)[-1]
        labels.append(activity)
        
        # identidy the separater of the csv file
        sep = sniffer.sniff(open(csv).read()).delimiter
        
        # error_bad_lines=False used to 
        if sep == ",":
            cur_instance = pd.read_csv(csv, skiprows=5, header=None, 
                                       error_bad_lines=False, warn_bad_lines=False)
        else:
            cur_instance = pd.read_csv(csv, skiprows=5, header=None, sep="\s+", 
                                       error_bad_lines=False, warn_bad_lines=False)
        
        # restore the column of csv file as we skip 5th row which is headers in the csv
        # however, they will becomes mass when separater is space. Thus, skip it
        cur_instance.columns = columns
        
        # break 6 time series into equal length, becomes 6*segs series
        seg_length = int(cur_instance.shape[0] / segs)
        segs_describe = []
        for i in range(segs):
            cur_seg = cur_instance[i * seg_length : min((i + 1) * seg_length, cur_instance.shape[0])]
            cur_describe = cur_seg.describe().drop('count').drop(columns="# Columns: time").T
            segs_describe.append(cur_describe.values.flatten())
        # complete current instance reading
        instances.append(np.concatenate(segs_describe, axis=None))
    
    # convert to dataframe and reorder the columns based on given feature list
    feature_df = pd.DataFrame(instances, columns=desc_order_idx)
    feature_df = feature_df.loc[:, feature]
    
    if standard:
        feature_df = feature_df.apply(lambda col: (col-col.mean()) / (np.std(col) + 1e-9), axis = 0)
    
    # add label to complete the dataset
    feature_df['label'] = pd.Series(labels)
    
    return feature_df
# reorder the columns
statistics = ['min','max','mean','median','std','_1stquart_', '_3rdquart_']
feature = [stat + str(i) for i in range(1, 7) for stat in statistics]

# load dataset as required
train_data = load_data(trainset, feature, columns)
test_data = load_data(testset, feature, columns)
test_data


,min1,max1,mean1,median1,std1,_1stquart_1,_3rdquart_1,min2,max2,mean2,...,_1stquart_5,_3rdquart_5,min6,max6,mean6,median6,std6,_1stquart_6,_3rdquart_6,label
0,37.25,45.00,40.624792,40.500,1.476967,39.2500,42.0000,0.0,1.30,0.358604,...,33.0000,36.0000,0.0,1.92,0.570583,0.430,0.582915,0.0000,1.300,bending1
1,38.00,45.67,42.812812,42.500,1.435550,42.0000,43.6700,0.0,1.22,0.372437,...,32.0000,34.5000,0.0,3.11,0.571083,0.430,0.601010,0.0000,1.300,bending1
2,19.33,43.50,34.227771,35.500,4.889576,30.5000,37.7500,0.0,14.50,3.995729,...,14.7500,18.6700,0.0,9.74,3.394125,3.100,1.792090,2.1050,4.425,walking
3,12.50,45.00,33.509729,34.125,4.850923,30.5000,36.7500,0.0,13.05,4.450771,...,14.6275,18.7500,0.0,8.96,3.378479,3.085,1.787360,2.0600,4.440,walking
4,15.00,46.75,34.660583,35.000,5.315110,31.0000,38.2500,0.0,13.44,4.200896,...,14.2500,18.5000,0.0,8.99,3.244396,3.000,1.630983,2.1200,4.240,walking
5,12.75,51.00,24.562958,24.250,3.737514,23.1875,26.5000,0.0,6.87,0.590833,...,20.5000,27.0000,0.0,4.97,0.700188,0.500,0.693720,0.4300,0.870,bending2
6,0.00,42.75,27.464604,28.000,3.583582,25.5000,30.0000,0.0,7.76,0.449708,...,15.0000,20.7500,0.0,6.76,1.122125,0.830,1.012342,0.4700,1.300,bending2
7,33.33,48.00,44.334729,45.000,2.476940,42.2500,46.5000,0.0,3.90,0.432958,...,9.3300,17.7500,0.0,5.02,0.933000,0.830,0.673609,0.4700,1.250,standing
8,35.50,46.25,43.174938,43.670,1.989052,42.5000,44.5000,0.0,2.12,0.506583,...,12.7500,16.5000,0.0,5.72,0.911979,0.830,0.666161,0.4700,1.220,standing
9,32.75,47.00,42.760562,44.500,3.398919,41.3300,45.3725,0.0,3.34,0.486167,...,13.0000,18.5650,0.0,5.73,0.842271,0.710,0.722165,0.4300,1.090,standing
